# Time Series Forcast - sunspots

```
Step 0. Library Import
Step 1. Load Dataset
Step 2. Data Preprocessing
Step 3. Modeling
Step 4. Model Compile
Step 5. Model Checkpoint
Step 6. Model Fit
Step 7. Model Evaluate & Save
Step 8. Model Reload
```

## Step 0. Library Import

In [1]:
import csv
import urllib
import numpy as np 

import tensorflow as tf 
from tensorflow.keras.layers import Dense, LSTM, Lambda, Conv1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import Huber

## Step 1. Load Dataset

In [2]:
url = 'https://storage.googleapis.com/download.tensorflow.org/data/Sunspots.csv'
data_path = './data/sunspots.csv'

urllib.request.urlretrieve(url, data_path)

('./data/sunspots.csv', <http.client.HTTPMessage at 0x7fb0853b8f90>)

In [3]:
sunspots = []
time_step = []

with open(data_path) as f:
    reader = csv.reader(f, delimiter=',')
    next(reader) # first line skip
    for row in reader:
        sunspots.append(float(row[2]))
        time_step.append(int(row[0]))

In [4]:
for i in range(10):
    print('sunspot : ', sunspots[i])
    print('time step :', time_step[i])
    print('-----------------------')

sunspot :  96.7
time step : 0
-----------------------
sunspot :  104.3
time step : 1
-----------------------
sunspot :  116.7
time step : 2
-----------------------
sunspot :  92.8
time step : 3
-----------------------
sunspot :  141.7
time step : 4
-----------------------
sunspot :  139.2
time step : 5
-----------------------
sunspot :  158.0
time step : 6
-----------------------
sunspot :  110.5
time step : 7
-----------------------
sunspot :  126.5
time step : 8
-----------------------
sunspot :  125.8
time step : 9
-----------------------


In [5]:
print('type of -sunspots- : ', type(sunspots))
print('type of -time_step- : ', type(time_step))

type of -sunspots- :  <class 'list'>
type of -time_step- :  <class 'list'>


In [6]:
sunspots = np.array(sunspots)
time_step = np.array(time_step)


In [7]:
print('type of -sunspots- : ', type(sunspots))
print('size of -sunspots- : ', sunspots.shape)
print('type of -time_step- : ', type(time_step))
print('size of -time_step- : ', time_step.shape)

type of -sunspots- :  <class 'numpy.ndarray'>
size of -sunspots- :  (3235,)
type of -time_step- :  <class 'numpy.ndarray'>
size of -time_step- :  (3235,)


## Step 2. Data Preprocessing

In [8]:
split_time = 3000

time_train = time_step[:split_time]
time_valid = time_step[split_time:]

x_train = sunspots[:split_time]
x_valid = sunspots[split_time:]

In [9]:
window_size = 30
batch_size = 32
shuffle_size = 1000

In [10]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size+1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w : w.batch(window_size+1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w : (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)

In [11]:
train_set = windowed_dataset(
    x_train,
    window_size = window_size,
    batch_size = batch_size,
    shuffle_buffer = shuffle_size
)

valid_set = windowed_dataset(
    x_valid,
    window_size = window_size,
    batch_size = batch_size,
    shuffle_buffer = shuffle_size
)

## Step 3. Modeling

In [12]:
model = Sequential([
    tf.keras.layers.Conv1D(60, kernel_size=5, padding='causal', activation='relu', input_shape=[None, 1]),
    tf.keras.layers.LSTM(60, return_sequences=True),
    tf.keras.layers.LSTM(60, return_sequences=True),
    tf.keras.layers.Dense(30, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1),
    tf.keras.layers.Lambda(lambda x : x * 400)
])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, None, 60)          360       
_________________________________________________________________
lstm (LSTM)                  (None, None, 60)          29040     
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 60)          29040     
_________________________________________________________________
dense (Dense)                (None, None, 30)          1830      
_________________________________________________________________
dense_1 (Dense)              (None, None, 10)          310       
_________________________________________________________________
dense_2 (Dense)              (None, None, 1)           11        
_________________________________________________________________
lambda (Lambda)              (None, None, 1)           0

## Step 4. Model Compile

In [14]:
optimizer = SGD(lr=1e-5, momentum=0.9)
loss = Huber()

In [15]:
model.compile(
    loss=loss,
    optimizer=optimizer,
    metrics=['mae']
)

## Step 5. Model Checkpoint

In [16]:
filepath = 'my_checkpoint.ckpt'
cp = ModelCheckpoint(
    filepath,
    save_weights_only=True,
    save_best_only=True,
    monitor='val_mae',
    verbose=1
    )

## Step 6. Model Fit

In [17]:
epochs = 30

In [18]:
model.fit(
    train_set,
    validation_data = valid_set,
    epochs=epochs,
    callbacks=[cp]
)

Epoch 1/30
     92/Unknown - 2s 23ms/step - loss: 42.3937 - mae: 42.8905
Epoch 00001: val_mae improved from inf to 20.35585, saving model to my_checkpoint.ckpt
93/93 [==============================] - 3s 32ms/step - loss: 42.2213 - mae: 42.7182 - val_loss: 19.8602 - val_mae: 20.3558
Epoch 2/30
91/93 [============================>.] - ETA: 0s - loss: 21.9800 - mae: 22.4733
Epoch 00002: val_mae improved from 20.35585 to 18.27837, saving model to my_checkpoint.ckpt
93/93 [==============================] - 2s 23ms/step - loss: 21.9348 - mae: 22.4282 - val_loss: 17.7844 - val_mae: 18.2784
Epoch 3/30
91/93 [============================>.] - ETA: 0s - loss: 19.6112 - mae: 20.1041
Epoch 00003: val_mae improved from 18.27837 to 16.02726, saving model to my_checkpoint.ckpt
93/93 [==============================] - 2s 23ms/step - loss: 19.6225 - mae: 20.1154 - val_loss: 15.5362 - val_mae: 16.0273
Epoch 4/30
91/93 [============================>.] - ETA: 0s - loss: 18.7598 - mae: 19.2522
Epoch 00004

## Step 7. Model Evaluate & Save

In [19]:
model.load_weights(filepath)

In [20]:
model.evaluate(valid_set)

7/7 [==============================] - 0s 5ms/step - loss: 13.8908 - mae: 14.3801


[13.890848159790039, 14.380138397216797]

In [21]:
model.save('./model/sunspot__val_mae_13.90.h5')

## Step 8. Model Reload

In [22]:
mymodel = tf.keras.models.load_model('./model/sunspot__val_mae_13.90.h5')

In [23]:
mymodel.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, None, 60)          360       
_________________________________________________________________
lstm (LSTM)                  (None, None, 60)          29040     
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 60)          29040     
_________________________________________________________________
dense (Dense)                (None, None, 30)          1830      
_________________________________________________________________
dense_1 (Dense)              (None, None, 10)          310       
_________________________________________________________________
dense_2 (Dense)              (None, None, 1)           11        
_________________________________________________________________
lambda (Lambda)              (None, None, 1)           0